In [1]:
import taichi as ti
from numpy import linspace, array
from scipy.interpolate import pchip_interpolate

[Taichi] version 1.7.2, llvm 15.0.4, commit 0131dce9, linux, python 3.11.8


[I 10/06/24 18:18:38.989 46697] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [4]:
import taichi as ti
import math

ti.init(arch=ti.cpu)

n = 400  # pixels of | orientation
aspect_ratio = 1
max_iterations = 300  # The more this value is, the figure is finer
pixels = ti.field(ti.f64, shape=(int(aspect_ratio*n), n))

@ti.func
def complex_sqr(z):
    return ti.Vector([z[0]**2 - z[1]**2, z[1] * z[0] * 2])

@ti.kernel
def paint(cx: ti.f64, cy: ti.f64, r:ti.f64):
    for i, j in pixels:  # Parallized over all pixels
        c = ti.Vector([-0.8, 0.2])
        z = ti.Vector([i / n * r - aspect_ratio*0.5*r + cx,
                       j / n * r - 0.5*r + cy])
        iterations = 0
        while z.norm() < 3:
            z = complex_sqr(z) + c
            iterations += 1
        pixels[i, j] = 1 - iterations / max_iterations


gui = ti.GUI("My Mandelbrot Set", res=(aspect_ratio*n, n))
steps = 100
cx_start, cx_end = -1, -1.31
cx_step = (cx_end - cx_start) / steps
r_start, r_end = 0.5, 0.12
r_step = (r_end - r_start) / steps
for i in range(steps+1):
    # distance from bound to center
    r = r_start + r_step * i
    # x coordinate of center
    cx = cx_start + cx_step*i
    # y coordinate of center
    cy = 0.
    paint(cx, cy, r)
    gui.set_image(pixels)
    # gui.show()
    gui.show("../frame/{:04d}.png".format(i))
gui.close()

[Taichi] Starting on arch=x64
